## Sudoku Cube Model

This Jupyter Notebook contains the data structures, randomizer, and visualizer for the Modeling the Sudoku Cube assignment of CS463G for Fall 2019. It requires numpy to run and the seed is set here to encourage consistent behaviour.

In [1]:
import numpy as np

np.random.seed(463)

The **data structure** for storing the cube consists of two classes: 

1. the `CubeFace` which stores two matrices specifying the value and orientation of the cells within the face, respectively, and defines getters and setters for the rows, columns, and overall state of the cube.


2. the `SudokuCube` class which consists of six CubeFace objects for each face of the cube, and defines methods for each of the twelve principle rotations of the sudoku cube. Note that only the clockwise rotations are independently specified, where the counterclockwise rotations are defined in terms of clockwise rotations. The class takes as a parameter the starting state of the cube which it then uses to initialize the cube faces.

**These data structures are run by passing a start state to the `SudokuCube` constructor and then calling the class methods to make moves on the cube face.**

The **randomizer** is implemented as a method of the `SudokuCube` class. It is called with a `num_turns` parameter which specifies the number of turns to randomly scramble the cube as well as optional flags to return the intermediate states and turn list of the randomization. It works by generating random numbers between 0 and 11 which are then mapped to one of the 12 move methods. If a move would just undo the previous move the number is rerolled.

**The randomizer is run by calling the `randomize` method of the `SudokuCube` class, passing as a parameter the `num_turns` that the cube should be randomized.**

In [2]:
class CubeFace:
    def __init__(self, value_matrix, orientation_matrix):
        self._val_mat = np.copy(np.array(value_matrix))
        self._ori_mat = np.copy(np.array(orientation_matrix))
        
    def get_state(self):
        return {
            "val": np.copy(self._val_mat),
            "ori": np.copy(self._ori_mat),
        }
    
    def set_state(self, value_matrix, ori_matrix):
        self._val_mat = np.copy(np.array(value_matrix))
        self._ori_mat = np.copy(np.array(ori_matrix))
    
    def get_left_column(self):
        return {
            "val": np.copy(self._val_mat[:, 0]),
            "ori": np.copy(self._ori_mat[:, 0]),
        }
    
    def set_left_column(self, value_vector, orientation_vector):
        self._val_mat[:, 0] = np.copy(value_vector)
        self._ori_mat[:, 0] = np.copy(orientation_vector)
    
    def get_right_column(self):
        return {
            "val": np.copy(self._val_mat[:, -1]),
            "ori": np.copy(self._ori_mat[:, -1]),
        }
    
    def set_right_column(self, value_vector, orientation_vector):
        self._val_mat[:, -1] = np.copy(value_vector)
        self._ori_mat[:, -1] = np.copy(orientation_vector)
    
    def get_top_row(self):
        return {
            "val": np.copy(self._val_mat[0, :]),
            "ori": np.copy(self._ori_mat[0, :]),
        }
    
    def set_top_row(self, value_vector, orientation_vector):
        self._val_mat[0, :] = np.copy(value_vector)
        self._ori_mat[0, :] = np.copy(orientation_vector)
    
    def get_bottom_row(self):
        return {
            "val": np.copy(self._val_mat[-1, :]),
            "ori": np.copy(self._ori_mat[-1, :]),
        }
    
    def set_bottom_row(self, value_vector, orientation_vector):
        self._val_mat[-1, :] = np.copy(value_vector)
        self._ori_mat[-1, :] = np.copy(orientation_vector)

In [3]:
class SudokuCube:
    def __init__(self, state_dict):
        self._front = CubeFace(state_dict["front_val"], state_dict["front_ori"])
        self._top = CubeFace(state_dict["top_val"], state_dict["top_ori"])
        self._bottom = CubeFace(state_dict["bottom_val"], state_dict["bottom_ori"])
        self._left = CubeFace(state_dict["left_val"], state_dict["left_ori"])
        self._right = CubeFace(state_dict["right_val"], state_dict["right_ori"])
        self._back = CubeFace(state_dict["back_val"], state_dict["back_ori"])

    def rotate_ori_clockwise(self, ori):
        up_mask = (ori == "^")
        down_mask = (ori == "v")
        left_mask = (ori == "<")
        right_mask = (ori == ">")
        
        ori[up_mask] = ">"
        ori[down_mask] = "<"
        ori[left_mask] = "^"
        ori[right_mask] = "v"
        
        return ori
    
    def rotate_ori_counterclockwise(self, ori):
        up_mask = (ori == "^")
        down_mask = (ori == "v")
        left_mask = (ori == "<")
        right_mask = (ori == ">")
        
        ori[up_mask] = "<"
        ori[down_mask] = ">"
        ori[left_mask] = "v"
        ori[right_mask] = "^"
        
        return ori
        
    def rotate_front_clockwise(self):
        front_state = self._front.get_state()
        
        new_front_val = np.rot90(front_state["val"], k=-1)
        new_front_ori = np.rot90(self.rotate_ori_clockwise(front_state["ori"]), k=-1)
        self._front.set_state(new_front_val, new_front_ori)
        
        top_row_state = self._top.get_bottom_row()
        right_column_state = self._right.get_left_column()
        bottom_row_state = self._bottom.get_top_row()
        left_column_state = self._left.get_right_column()
        
        new_right_column_val = top_row_state["val"]
        new_right_column_ori = self.rotate_ori_clockwise(top_row_state["ori"])
        self._right.set_left_column(new_right_column_val, new_right_column_ori)
        
        new_bottom_row_val = np.flip(right_column_state["val"])
        new_bottom_row_ori = np.flip(self.rotate_ori_clockwise(right_column_state["ori"]))
        self._bottom.set_top_row(new_bottom_row_val, new_bottom_row_ori)
        
        new_left_column_val = bottom_row_state["val"]
        new_left_column_ori = self.rotate_ori_clockwise(bottom_row_state["ori"])
        self._left.set_right_column(new_left_column_val, new_left_column_ori)
        
        new_top_row_val = np.flip(left_column_state["val"])
        new_top_row_ori = np.flip(self.rotate_ori_clockwise(left_column_state["ori"]))
        self._top.set_bottom_row(new_top_row_val, new_top_row_ori)
    
    def rotate_front_counterclockwise(self):
        for _ in range(3):
            self.rotate_front_clockwise()
    
    def rotate_top_clockwise(self):
        top_state = self._top.get_state()
        
        new_top_val = np.rot90(top_state["val"], k=-1)
        new_top_ori = np.rot90(self.rotate_ori_clockwise(top_state["ori"]), k=-1)
        self._top.set_state(new_top_val, new_top_ori)
        
        front_row_state = self._front.get_top_row()
        right_row_state = self._right.get_top_row()
        back_row_state = self._back.get_bottom_row()
        left_row_state = self._left.get_top_row()
        
        new_right_row_val = np.flip(back_row_state["val"])
        new_right_row_ori = np.flip(self.rotate_ori_clockwise(self.rotate_ori_clockwise(back_row_state["ori"])))
        self._right.set_top_row(new_right_row_val, new_right_row_ori)
        
        new_left_row_val = front_row_state["val"]
        new_left_row_ori = front_row_state["ori"]
        self._left.set_top_row(new_left_row_val, new_left_row_ori)
        
        new_front_row_val = right_row_state["val"]
        new_front_row_ori = right_row_state["ori"]
        self._front.set_top_row(new_front_row_val, new_front_row_ori)
        
        new_back_row_val = np.flip(left_row_state["val"])
        new_back_row_ori = np.flip(self.rotate_ori_clockwise(self.rotate_ori_clockwise(left_row_state["ori"])))
        self._back.set_bottom_row(new_back_row_val, new_back_row_ori)
    
    def rotate_top_counterclockwise(self):
        for _ in range(3):
            self.rotate_top_clockwise()
    
    def rotate_bottom_clockwise(self):
        bottom_state = self._bottom.get_state()
        
        new_bottom_val = np.rot90(bottom_state["val"], k=-1)
        new_bottom_ori = np.rot90(self.rotate_ori_clockwise(bottom_state["ori"]), k=-1)
        self._bottom.set_state(new_bottom_val, new_bottom_ori)
        
        front_row_state = self._front.get_bottom_row()
        right_row_state = self._right.get_bottom_row()
        back_row_state = self._back.get_top_row()
        left_row_state = self._left.get_bottom_row()
        
        new_right_row_val = front_row_state["val"]
        new_right_row_ori = front_row_state["ori"]
        self._right.set_bottom_row(new_right_row_val, new_right_row_ori)
        
        new_left_row_val = np.flip(back_row_state["val"])
        new_left_row_ori = np.flip(self.rotate_ori_clockwise(self.rotate_ori_clockwise(back_row_state["ori"])))
        self._left.set_bottom_row(new_left_row_val, new_left_row_ori)
        
        new_front_row_val = left_row_state["val"]
        new_front_row_ori = left_row_state["ori"]
        self._front.set_bottom_row(new_front_row_val, new_front_row_ori)
        
        new_back_row_val = np.flip(right_row_state["val"])
        new_back_row_ori = np.flip(self.rotate_ori_clockwise(self.rotate_ori_clockwise(right_row_state["ori"])))
        self._back.set_top_row(new_back_row_val, new_back_row_ori)
    
    def rotate_bottom_counterclockwise(self):
        for _ in range(3):
            self.rotate_bottom_clockwise()
    
    def rotate_left_clockwise(self):
        left_state = self._left.get_state()
        
        new_left_val = np.rot90(left_state["val"], k=-1)
        new_left_ori = np.rot90(self.rotate_ori_clockwise(left_state["ori"]), k=-1)
        self._left.set_state(new_left_val, new_left_ori)
        
        front_column_state = self._front.get_left_column()
        top_column_state = self._top.get_left_column()
        back_column_state = self._back.get_left_column()
        bottom_column_state = self._bottom.get_left_column()
        
        new_front_column_val = top_column_state["val"]
        new_front_column_ori = top_column_state["ori"]
        self._front.set_left_column(new_front_column_val, new_front_column_ori)
        
        new_top_column_val = back_column_state["val"]
        new_top_column_ori = back_column_state["ori"]
        self._top.set_left_column(new_top_column_val, new_top_column_ori)
        
        new_back_column_val = bottom_column_state["val"]
        new_back_column_ori = bottom_column_state["ori"]
        self._back.set_left_column(new_back_column_val, new_back_column_ori)
        
        new_bottom_column_val = front_column_state["val"]
        new_bottom_column_ori = front_column_state["ori"]
        self._bottom.set_left_column(new_bottom_column_val, new_bottom_column_ori)
    
    def rotate_left_counterclockwise(self):
        for _ in range(3):
            self.rotate_left_clockwise()
    
    def rotate_right_clockwise(self):
        right_state = self._right.get_state()
        
        new_right_val = np.rot90(right_state["val"], k=-1)
        new_right_ori = np.rot90(self.rotate_ori_clockwise(right_state["ori"]), k=-1)
        self._right.set_state(new_right_val, new_right_ori)
        
        front_column_state = self._front.get_right_column()
        top_column_state = self._top.get_right_column()
        back_column_state = self._back.get_right_column()
        bottom_column_state = self._bottom.get_right_column()
        
        new_front_column_val = bottom_column_state["val"]
        new_front_column_ori = bottom_column_state["ori"]
        self._front.set_right_column(new_front_column_val, new_front_column_ori)
        
        new_top_column_val = front_column_state["val"]
        new_top_column_ori = front_column_state["ori"]
        self._top.set_right_column(new_top_column_val, new_top_column_ori)
        
        new_back_column_val = top_column_state["val"]
        new_back_column_ori = top_column_state["ori"]
        self._back.set_right_column(new_back_column_val, new_back_column_ori)
        
        new_bottom_column_val = back_column_state["val"]
        new_bottom_column_ori = back_column_state["ori"]
        self._bottom.set_right_column(new_bottom_column_val, new_bottom_column_ori)
    
    def rotate_right_counterclockwise(self):
        for _ in range(3):
            self.rotate_right_clockwise()
    
    def rotate_back_clockwise(self):
        back_state = self._back.get_state()
        
        new_back_val = np.rot90(back_state["val"], k=1)
        new_back_ori = np.rot90(self.rotate_ori_counterclockwise(back_state["ori"]), k=1)
        self._back.set_state(new_back_val, new_back_ori)
        
        top_row_state = self._top.get_top_row()
        right_column_state = self._right.get_right_column()
        bottom_row_state = self._bottom.get_bottom_row()
        left_column_state = self._left.get_left_column()
        
        new_right_column_val = top_row_state["val"]
        new_right_column_ori = self.rotate_ori_clockwise(top_row_state["ori"])
        self._right.set_right_column(new_right_column_val, new_right_column_ori)
        
        new_bottom_row_val = np.flip(right_column_state["val"])
        new_bottom_row_ori = np.flip(self.rotate_ori_clockwise(right_column_state["ori"]))
        self._bottom.set_bottom_row(new_bottom_row_val, new_bottom_row_ori)
        
        new_left_column_val = bottom_row_state["val"]
        new_left_column_ori = self.rotate_ori_clockwise(bottom_row_state["ori"])
        self._left.set_left_column(new_left_column_val, new_left_column_ori)
        
        new_top_row_val = np.flip(left_column_state["val"])
        new_top_row_ori = np.flip(self.rotate_ori_clockwise(left_column_state["ori"]))
        self._top.set_top_row(new_top_row_val, new_top_row_ori)
    
    def rotate_back_counterclockwise(self):
        for _ in range(3):
            self.rotate_back_clockwise()
            
    def ensure_non_switchback_move(self, move_number, previous_move_number):
        if previous_move_number == 0 and move_number == 1:
            while move_number == 1:
                move_number = np.random.randint(12)
        
        elif previous_move_number == 1 and move_number == 0:
            while move_number == 0:
                move_number = np.random.randint(12)
                
        elif previous_move_number == 2 and move_number == 3:
            while move_number == 3:
                move_number = np.random.randint(12)
                
        elif previous_move_number == 3 and move_number == 2:
            while move_number == 2:
                move_number = np.random.randint(12)
                
        elif previous_move_number == 4 and move_number == 5:
            while move_number == 5:
                move_number = np.random.randint(12)
                
        elif previous_move_number == 5 and move_number == 4:
            while move_number == 4:
                move_number = np.random.randint(12)
                
        elif previous_move_number == 6 and move_number == 7:
            while move_number == 7:
                move_number = np.random.randint(12)
                
        elif previous_move_number == 7 and move_number == 6:
            while move_number == 6:
                move_number = np.random.randint(12)
                
        elif previous_move_number == 8 and move_number == 9:
            while move_number == 9:
                move_number = np.random.randint(12)
                
        elif previous_move_number == 9 and move_number == 8:
            while move_number == 8:
                move_number = np.random.randint(12)
                
        elif previous_move_number == 10 and move_number == 11:
            while move_number == 11:
                move_number = np.random.randint(12)
                
        elif previous_move_number == 11 and move_number == 10:
            while move_number == 10:
                move_number = np.random.randint(12)
                
        return move_number
    
    def randomize(self, num_turns, return_state_sequence=False, return_move_sequence=False):
        state_sequence = [self.get_cube_state()]
        move_sequence = []
        previous_move_number = -1
        
        for move_number in np.random.randint(12, size=num_turns):
            move_number = self.ensure_non_switchback_move(move_number, previous_move_number)
            if move_number == 0:
                self.rotate_front_clockwise()
                move_sequence.append("F")
                
            elif move_number == 1:
                self.rotate_front_counterclockwise()
                move_sequence.append("F'")
                
            elif move_number == 2:
                self.rotate_top_clockwise()
                move_sequence.append("T")
                
            elif move_number == 3:
                self.rotate_top_counterclockwise()
                move_sequence.append("T'")
                
            elif move_number == 4:
                self.rotate_bottom_clockwise()
                move_sequence.append("B")
                
            elif move_number == 5:
                self.rotate_bottom_counterclockwise()
                move_sequence.append("B'")
                
            elif move_number == 6:
                self.rotate_left_clockwise()
                move_sequence.append("L")
                
            elif move_number == 7:
                self.rotate_left_counterclockwise()
                move_sequence.append("L'")
                
            elif move_number == 8:
                self.rotate_right_clockwise()
                move_sequence.append("R")
                
            elif move_number == 9:
                self.rotate_right_counterclockwise()
                move_sequence.append("R'")
                
            elif move_number == 10:
                self.rotate_back_clockwise()
                move_sequence.append("Ba")
                
            elif move_number == 11:
                self.rotate_back_counterclockwise()
                move_sequence.append("Ba'")
                
            state_sequence.append(self.get_cube_state())
            previous_move_number = move_number
            
        if return_state_sequence and return_move_sequence:
            return state_sequence, move_sequence
        
        elif return_state_sequence:
            return state_sequence
        
        elif return_move_sequence:
            return move_sequence
        
    def get_cube_state(self):
        front_state = self._front.get_state()
        top_state = self._top.get_state()
        bottom_state = self._bottom.get_state()
        left_state = self._left.get_state()
        right_state = self._right.get_state()
        back_state = self._back.get_state()
        
        cube_state = {
            "front_val": front_state["val"],
            "front_ori": front_state["ori"],
            "top_val": top_state["val"],
            "top_ori": top_state["ori"],
            "bottom_val": bottom_state["val"],
            "bottom_ori": bottom_state["ori"],
            "left_val": left_state["val"],
            "left_ori": left_state["ori"],
            "right_val": right_state["val"],
            "right_ori": right_state["ori"],
            "back_val": back_state["val"],
            "back_ori": back_state["ori"],
        }
        
        return cube_state

The **GUI Implementation** is done with the `CubeVisualizer` class. This class takes as an argument a sequence of cube states and prints to the console a cross pattern representation of the sudoku cube. Optionally a list of moves used to get between states can be specified and value/orientation maps can be provided to alter the appearance of the output.

In [4]:
class CubeVisualizer:
    def __init__(self, state_sequence, move_sequence=None, value_map=None, orientation_map=None):
        self._state_sequence = state_sequence
        self._value_map = value_map
        self._orientation_map = orientation_map
        self._move_sequence = move_sequence
    
    def visualize(self):
        if self._move_sequence:
            move_counter = 0
        
        for state in self._state_sequence:
            text_matrix = np.full((15, 30), fill_value= " ")
            text_matrix[0:3, 4:7] = state["back_val"]
            text_matrix[4:7, 4:7] = state["top_val"]
            text_matrix[8:11, 4:7] = state["front_val"]
            text_matrix[12:15, 4:7] = state["bottom_val"]
            text_matrix[8:11, 0:3] = state["left_val"]
            text_matrix[8:11, 8:11] = state["right_val"]
            
            text_matrix[0:3, 23:26] = state["back_ori"]
            text_matrix[4:7, 23:26] = state["top_ori"]
            text_matrix[8:11, 23:26] = state["front_ori"]
            text_matrix[12:15, 23:26] = state["bottom_ori"]
            text_matrix[8:11, 19:22] = state["left_ori"]
            text_matrix[8:11, 27:30] = state["right_ori"]
            
            for row in text_matrix:
                text_string = "".join(row.tolist())
                if self._value_map:
                    for key, val in self._value_map.items():
                        text_string = text_string.replace(key, val)
                        
                if self._orientation_map:
                    for key, val in self._orientation_map.items():
                        text_string = text_string.replace(key, val)
                
                print(text_string)
                
            print("\n")
            if self._move_sequence and move_counter != len(self._move_sequence):
                print("Move: " + self._move_sequence[move_counter])
                move_counter += 1
            print("\n")
            

These dictionaries specify the starting state of the cube, as well as provide example value maps for altering the GUI output.

In [5]:
start_state = {
    "front_val": np.array([
        [1, 5, 2],
        [4, 6, 3],
        [7, 8, 9],
    ]),
    "front_ori": np.array([
        ["^", "^", "^"],
        ["^", "^", "^"],
        ["^", "^", "^"],
    ]),
    "top_val": np.array([
        [9, 7, 1],
        [2, 4, 8],
        [5, 3, 6],
    ]),
    "top_ori": np.array([
        ["^", "^", "^"],
        ["^", "^", "^"],
        ["^", "^", "^"],
    ]),
    "bottom_val": np.array([
        [5, 9, 6],
        [3, 7, 8],
        [2, 4, 1],
    ]),
    "bottom_ori": np.array([
        ["v", "v", "v"],
        ["v", "v", "v"],
        ["v", "v", "v"],
    ]),
    "left_val": np.array([
        [3, 8, 7],
        [2, 1, 9],
        [5, 6, 4],
    ]),
    "left_ori": np.array([
        ["^", "^", "^"],
        ["^", "^", "^"],
        ["^", "^", "^"],
    ]),
    "right_val": np.array([
        [8, 6, 3],
        [2, 1, 9],
        [4, 5, 7],
    ]),
    "right_ori": np.array([
        [">", ">", ">"],
        [">", ">", ">"],
        [">", ">", ">"],
    ]),
    "back_val": np.array([
        [8, 2, 9],
        [1, 5, 7],
        [6, 3, 4],
    ]),
    "back_ori": np.array([
        ["v", "v", "v"],
        ["v", "v", "v"],
        ["v", "v", "v"],
    ]),
}

value_map = {
    "1": "💩",
    "2": "💯",
    "3": "❤️",
    "4": "🧡",
    "5": "💛",
    "6": "💚",
    "7": "💙",
    "8": "💜",
    "9": "🖤",
    " ": "🍞"
}

orientation_map = {
    "^": "⬆️",
    ">": "➡️",
    "<": "⬅️",
    "v": "⬇️",
}

This example **demonstrates the GUI** first with a simple randomization sequence then with a shorter randomization with modified face characters.

In [6]:
import sys

if __name__ == "__main__":
    test_cube = SudokuCube(start_state)
    
    # Case where running the script from the command line
    if len(sys.argv) == 2 and sys.argv[0].endswith(".py"):
        states, moves = test_cube.randomize(sys.argv[1], return_state_sequence=True, return_move_sequence=True)
        
        viz = CubeVisualizer(states, moves)
        viz.visualize()
        
    # Jupyter notebook example
    else:
        test_cube = SudokuCube(start_state)
        states, moves = test_cube.randomize(5, return_state_sequence=True, return_move_sequence=True)
        
        viz = CubeVisualizer(states, moves)
        viz.visualize()
        
        print("BEHOLD: THE HORROR AND GLORY OF THE EMOJI CUBE\n\n")
        
        test_cube = SudokuCube(start_state)
        states, moves = test_cube.randomize(2, return_state_sequence=True, return_move_sequence=True)
        
        viz = CubeVisualizer(states, moves, value_map, orientation_map)
        viz.visualize()

    829                vvv    
    157                vvv    
    634                vvv    
                              
    971                ^^^    
    248                ^^^    
    536                ^^^    
                              
387 152 863        ^^^ ^^^ >>>
219 463 219        ^^^ ^^^ >>>
564 789 457        ^^^ ^^^ >>>
                              
    596                vvv    
    378                vvv    
    241                vvv    


Move: R


    821                vv^    
    158                vv^    
    636                vv^    
                              
    972                ^^^    
    243                ^^^    
    539                ^^^    
                              
387 156 428        ^^^ ^^v vvv
219 468 516        ^^^ ^^v vvv
564 781 793        ^^^ ^^v vvv
                              
    599                vvv    
    377                vvv    
    244                vvv    


Move: F


    821                vv^    
    158        

## Heuristic

My heuristic for A* is the total number of locally invalid rows and columns in the sudoku cube (i.e. a count of each value row/column with duplicate numbers and each orientation row/column that is not all identical). There are 72 such rows/columns in both the orientation and value cubes combined and up to 32 of them can be changed in a single move (four on each face, four faces affected by a rotation, two cubes = 4 * 4 * 2 = 32). So the final heuristic is **number of improper rows/columns divided by 32**. This value can span from 0 at the solved state to 2.25 when every row/column is out of order. This heuristic is **admissible** because the division normalizes it to count the maximum number of corrections in a turn as a value of 1 move. This is easy to compute by simply checking each row/column at a cube state, but is very bad as a heuristic because the value range is so small.

## What I Learned

In this assignment I learned two primary lessons:

1. How difficult it is to come up with a heuristic that is both useful and admissable.

2. How difficult it is to properly code and mentally visualize a 3D object with rotations around that object when unfolded into the 2D plane.